[BIOBB](https://mmb.irbbarcelona.org/biobb/workflows/tutorials/ligand-parameterization)

In [ ]:
# @title # 📁👻 **Ruta de trabajo**
from google.colab import drive
drive.mount('/content/drive')
ruta_carpeta_drive = "Colab_Notebooks/Chile" #@param {type:"string"}
carpeta = "/content/drive/MyDrive/" + ruta_carpeta_drive
%cd {carpeta}

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Chile


In [ ]:
# @title # 0️⃣👻 **Recursos necesarios para el proceso**
!pip install nglview==3.0.8
!pip install "ipywidgets>=7,<8"
!pip install biobb_io
!pip install Bio
!pip install biobb_chemistry
!apt install openbabel
from google.colab import output
output.enable_custom_widget_manager()
import nglview
import ipywidgets
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
from biobb_io.api.ligand import ligand
from biobb_chemistry.babelm.babel_add_hydrogens import babel_add_hydrogens
from biobb_chemistry.babelm.babel_minimize import babel_minimize
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.2 MB/s eta 0:00:00
  Created wheel for nglview: filename=nglview-3.0.8-py3-none-any.whl size=10216160 sha256=6e4734db81cf97b7fd34b236a26ef303c9cb1feeb241cb66fe9e179bed5775e0
  Stored in directory: /root/.cache/pip/wheels/2e/6c/59/32bf4aa0134f9c4cdca054f5192839fb4285241e2f17f7d358
Successfully built nglview
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 220.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done

In [ ]:
# @title # 1️⃣👻 **Ligando(s)**
# @markdown #  1️⃣ **Carga de información de ligando(s)**
# @markdown ---
# @markdown ## 🅰 o 🅱️ **Trabajar con ID de PDB o con archivo PDB**
# @markdown #### 👀 **Nota:** Recuerde si escoje **Archivo_PDB** debe tener los archivos en la carpeta de Drive que va a trabajar
formato_a_trabajar = "Archivo_PDB" #@param ["ID_PDBDATE", "Archivo_PDB"]
# @markdown ---
# @markdown ## 2️⃣ **Nombre del ID o del archivo**
# @markdown #### 👀 **Nota 1:** Separe los nombres con **espacios**
# @markdown #### 👀 **Nota 2:** Si va a trabajar con archivo(s) no agregue la extensión **.pdb o .mol2.** La extensión del archivo se selecciona


lista_ligando = "amino imino IBB" #@param {type:"string"}
lista_ligando = lista_ligando.split(" ")
if formato_a_trabajar=="ID_PDBDATE":
  for ligando in lista_ligando:
    ligando_pdb = ligando + '.pdb'
    prop = {
          'ligand_code' : ligando
          }
    ligand(output_pdb_path=ligando_pdb,
          properties=prop)
formato_archivo = "pdb" #@param ["pdb", "mol2", "sdf"]
# @markdown ---
# @markdown ## 3️⃣ **Carga de ligando(s)**
# @markdown #### 👀 **Nota 1:** Separe los nombres con **espacios**
# @markdown #### 👀 **Nota 2:** Se debe agregar en el mismo orden que colocaron los ligandos

carga_ligando = "0" #@param {type:"string"}
carga_ligando = carga_ligando.split(" ")
!rm -f log*

In [ ]:
# @title # 2️⃣👻 **Agregar hidrogenos a determinado pH más minimización**
# @markdown ## 1️⃣ 🅰 o 🅱️ **Trabajar con un solo pH o trabajar con varios pH**
# @markdown #### 👀 **Nota:** Separe los valores de pH con **espacios**, si surge un error puede ser por un **doble espacio**
valor_pH = "1 7 14" #@param {type:"string"}
valor_pH = valor_pH.split(" ")
# @markdown ## 2️⃣ **Realizar o no Minimización de energía**
minimizacion_energia_ligando = False # @param ["False", "True"] {type:"raw"}
# @markdown #### 👻 **Parámetros**
# @markdown ##### 🅰️ **Método:** **cg** (algoritmo de gradientes conjugados), **sd** (algoritmo de descenso más pronunciado)
metodo_minimizacion = "sd" #@param ["sd","cg"]
# @markdown ##### 🅱️ **Campo de fuerza**
campo_de_fuerza = "MMFF94" #@param ["GAFF","GAFF2","Ghemical","MMFF94","MMFF94s","UFF"]
# @markdown ##### 🅾️ **Criterios de convergencia:** Por defecto deje el valor **1e-6**
criterio_convergencia = "1e-6" #@param {type:"string"}

if minimizacion_energia_ligando == True:
  for ligando in lista_ligando:
    for pH in valor_pH:
      pH=float(pH)
      entrada_ligando=ligando + '.' + formato_archivo
      salida_babel_h = ligando + str(pH) + '.H.mol2'
      !obabel {entrada_ligando} -O {salida_babel_h} -p {pH}
      salida_babel_min = ligando + str(pH) + '.H.min.pdb'
      prop = {
          'method' : metodo_minimizacion,
          'criteria' : criterio_convergencia,
          'force_field' : campo_de_fuerza
      }

      babel_minimize(input_path=salida_babel_h,
                    output_path=salida_babel_min,
                    properties=prop)
elif minimizacion_energia_ligando == False:
  for ligando in lista_ligando:
    for pH in valor_pH:
      pH=float(pH)
      entrada_ligando=ligando + '.' + formato_archivo
      salida_babel_h = ligando + str(pH) + '.H.mol2'
      !obabel {entrada_ligando} -O {salida_babel_h} -p {pH}

!rm -f log*

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is amino.pdb)

1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is amino.pdb)

1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is amino.pdb)

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


In [ ]:
# @title # 3️⃣📉 **gráfica de de estructuras**
if minimizacion_energia_ligando == True:
  views = []
  for ligando in lista_ligando:
    view = nglview.show_structure_file(ligando + "." + formato_archivo)
    view.add_representation(repr_type='ball+stick')
    view._remote_call('setSize', target='Widget', args=['250px','300px'])
    view.camera='orthographic'
    views.append(view)
    for pH in valor_pH:
      nombre = ligando + str(float(pH)) + ".H.min.pdb"
      view = nglview.show_structure_file(nombre)
      view.add_representation(repr_type='ball+stick')
      view._remote_call('setSize', target='Widget', args=['250px','300px'])
      view.camera='orthographic'
      views.append(view)
if minimizacion_energia_ligando == False:
  views = []
  for ligando in lista_ligando:
    view = nglview.show_structure_file(ligando + "." + formato_archivo)
    view.add_representation(repr_type='ball+stick')
    view._remote_call('setSize', target='Widget', args=['250px','300px'])
    view.camera='orthographic'
    views.append(view)
    for pH in valor_pH:
      nombre = ligando + str(float(pH)) + ".H.mol2"
      view = nglview.show_structure_file(nombre)
      view.add_representation(repr_type='ball+stick')
      view._remote_call('setSize', target='Widget', args=['250px','300px'])
      view.camera='orthographic'
      views.append(view)

numero_filas = len(lista_ligando)
if len(valor_pH) > 4:
  numero_columnas = 5
else:
  numero_columnas = str(len(valor_pH)+1)

ipywidgets.GridBox(views, layout=ipywidgets.Layout(grid_template_columns=f"repeat({numero_columnas}, 0fr)", grid_template_rows=f"repeat({numero_filas}, 0fr)"))

GridBox(children=(NGLWidget(), NGLWidget(), NGLWidget(), NGLWidget(), NGLWidget(), NGLWidget(), NGLWidget(), N…

# Preparar archivos para dinamica molecular:  Gromacs o Amber

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda update -n base -c conda-forge conda
!conda init
!conda create --name AmberTools23
!conda activate AmberTools23
!conda install -c conda-forge ambertools=23
!conda update -c conda-forge ambertools
!pip install acpype
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

In [ ]:
output_acpype_gro = ligandCode + 'params.gro'
output_acpype_itp = ligandCode + 'params.itp'
output_acpype_top = ligandCode + 'params.top'
output_acpype = ligandCode + 'params'
prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}
acpype_params_gmx(input_path=output_babel_min,
                output_path_gro=output_acpype_gro,
                output_path_itp=output_acpype_itp,
                output_path_top=output_acpype_top,
                properties=prop)